In [16]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIXED #####
# Updated the import to match your CRUD Python module file name and class name
from ModuleJosephDengler import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIXED: Updated with your username and password
username = "aacuser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelter()

# Retrieve all records from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Drop the '_id' column to avoid data_table crash
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIXED: Add in Grazioso Salvare’s logo and your unique identifier
image_filename = 'Grazioso Salvare Logo.png'  # replace with the correct path to your logo image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Joseph Dengler'))),
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())), href='http://www.snhu.edu')),
    html.Hr(),
    html.Div([
        # FIXED: Added in code for the interactive filtering options (Radio buttons, drop down, checkboxes, etc.)
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_wilderness_rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster_individual_tracking'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         # FIXED: Set up the features for your interactive data table
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         page_action="native",
                         page_current=0,
                         page_size=10,
                         style_table={'overflowX': 'auto'},
                         style_cell={
                             'height': 'auto',
                             'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
                             'whiteSpace': 'normal'
                         },
                         row_selectable='single',
                         selected_rows=[0],
                         ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # FIXED: Add code to filter interactive data table with MongoDB queries
    query = {}
    if filter_type == 'water_rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", "Newfoundland"]}}
    elif filter_type == 'mountain_wilderness_rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'disaster_individual_tracking':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    
    data = pd.DataFrame.from_records(db.read(query))
    data.drop(columns=['_id'], inplace=True)
    return data.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # FIXED: Added code for a pie chart displaying breed distribution
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available to display.")
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Animals by Breed')
    return [
        dcc.Graph(
            figure=fig
        )
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run app and display result inline in the notebook
app.run_server(debug=True)

Connection to MongoDB successful!
Query returned 9999 documents.
Dash app running on http://127.0.0.1:28583/
Query returned 9999 documents.
Query returned 18 documents.
Query returned 47 documents.
Query returned 36 documents.
Query returned 9999 documents.
